In [25]:
import omdb
import sqlite3
import pandas as pd
import os
import requests
import shutil
import io
import PIL.Image as Image

# first read in OMDB python key
with open('api_key') as f:
    lines = f.readlines()
api_key = lines[0]
omdb.set_default('apikey', api_key)


In [26]:
# all series
st_series = {'tos': 'tt0060028',
             'tas': 'tt0069637',
             'tng': 'tt0092455',
             'ds9': 'tt0106145',
             'voy': 'tt0112178',
             'ent': 'tt0244365',
             'dis': 'tt5171438',
             'sho': 'tt9059594',
             'pic': 'tt8806524',
             'lds': 'tt9184820',
             'pro': 'tt9795876',
             'snw': 'tt12327578',
             } 

## Series Request

In [27]:
# delete current database if exists
db_filename = "treklist.db"
#if os.path.exists(db_filename):
#    os.remove(db_filename)

# make omdb requests for series
conn = sqlite3.connect(db_filename)
curs = conn.cursor()

#conn.execute("DROP TABLE IF EXISTS series")
conn.execute("CREATE TABLE IF NOT EXISTS series (abb TEXT, title TEXT, imdb_id TEXT, year TEXT, total_seasons INTEGER, poster_url TEXT, poster BLOB, rated TEXT)")
for key in st_series.keys():
    res = omdb.imdbid(st_series[key])

    # check to see if series already exists
    cmd = f"SELECT * from series WHERE imdb_id == '{res['imdb_id']}'"
    df = pd.read_sql_query(cmd, conn)

    # add it
    if len(df) == 0:
        cmd = "INSERT INTO series (abb, title, imdb_id, year, total_seasons, poster_url, rated) VALUES (?, ?, ?, ?, ?, ?, ?) "
        curs.execute(cmd, [key, res['title'], res['imdb_id'], res['year'], res['total_seasons'], res['poster'], res['rated']])
        conn.commit()

df = pd.read_sql_query("SELECT * FROM series", conn)
df


,abb,title,imdb_id,year,total_seasons,poster_url,poster,rated
0,tos,Star Trek,tt0060028,1966–1969,3,https://m.media-amazon.com/images/M/MV5BNDRkMT...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,TV-PG
1,tas,Star Trek: The Animated Series,tt0069637,1973–1975,2,https://m.media-amazon.com/images/M/MV5BMzI0Y2...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,TV-Y7
2,tng,Star Trek: The Next Generation,tt0092455,1987–1994,7,https://m.media-amazon.com/images/M/MV5BOWFhYj...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,TV-PG
3,ds9,Star Trek: Deep Space Nine,tt0106145,1993–1999,7,https://m.media-amazon.com/images/M/MV5BMDc3OG...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,TV-PG
4,voy,Star Trek: Voyager,tt0112178,1995–2001,7,https://m.media-amazon.com/images/M/MV5BYWIwMT...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,TV-PG
5,ent,Star Trek: Enterprise,tt0244365,2001–2005,4,https://m.media-amazon.com/images/M/MV5BODg3Zm...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,TV-PG
6,dis,Star Trek: Discovery,tt5171438,2017–,5,https://m.media-amazon.com/images/M/MV5BNjg1NT...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,TV-14
7,sho,Star Trek: Short Treks,tt9059594,2018–2020,2,https://m.media-amazon.com/images/M/MV5BNWY2NW...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,TV-PG
8,pic,Star Trek: Picard,tt8806524,2020–,3,https://m.media-amazon.com/images/M/MV5BMmMzNT...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,TV-MA
9,lds,Star Trek: Lower Decks,tt9184820,2020–,3,https://m.media-amazon.com/images/M/MV5BNjZkNG...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,TV-14


## Episodes Request

In [28]:
keys = list(st_series.keys())
for key in keys: # temporary for testing
    conn.execute(f"CREATE TABLE IF NOT EXISTS {key} (title TEXT, rated TEXT, released DATE, season INTEGER, episode INTEGER, runtime TEXT, director TEXT, writer TEXT, actors TEXT, plot TEXT, poster_url TEXT, poster BLOB, imdb_rating FLOAT, imdb_votes INTEGER, imdb_id TEXT)")

    tot_seasons = df['total_seasons'][df['abb'] == key].values[0]
    print('%s seasons %i' % (key, tot_seasons))
    for season in range(tot_seasons):# another temporary....
        season += 1 # account for zero index
        res = omdb.imdbid(st_series[key], season=season)
        for ep in res['episodes']:

            # check if it is already in the database
            df_check = pd.read_sql_query(f"SELECT * FROM {key} WHERE imdb_id = '{ep['imdb_id']}'", conn)
            ep_exists = True if len(df_check) > 0 else False
            if not ep_exists:
                ep_info = omdb.imdbid(ep['imdb_id'])
                cmd = f"INSERT INTO {key} (title, rated, released, season, episode, runtime, director, writer, actors, plot, poster_url, imdb_rating, imdb_votes, imdb_id) VALUES " + "(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
                curs.execute(cmd, [ep_info['title'], ep_info['rated'], ep['released'], season, ep_info['episode'], ep_info['runtime'], ep_info['director'], ep_info['writer'], ep_info['actors'], ep_info['plot'], ep_info['poster'], ep_info['imdb_rating'], ep_info['imdb_votes'], ep_info['imdb_id']])
    conn.commit()

tos seasons 3
tas seasons 2
tng seasons 7
ds9 seasons 7
voy seasons 7
ent seasons 4
dis seasons 5
sho seasons 2
pic seasons 3
lds seasons 3
pro seasons 1
snw seasons 1


## Download/Insert Posters

In [29]:
# Download posters for series only
df = pd.read_sql_query(f"SELECT * from series", conn)
for i, row in df.iterrows():
    poster_url = row['poster_url']
    if poster_url != "N/A":
        # insert poster
        img = requests.get(row['poster_url'], stream=True)
        cmd = f"UPDATE series SET poster = ? WHERE imdb_id = '{row['imdb_id']}'"
        curs.execute(cmd, [sqlite3.Binary(img.content)])
        conn.commit()

In [6]:
# optional - read back a poster
#curs.execute(f"SELECT * FROM series WHERE imdb_id = 'tt0060028'")
#record = curs.fetchall()
#idx = df.keys().to_list().index('poster')
#image = Image.open(io.BytesIO(record[0][idx]))
#image.show()

In [30]:
# download poster for individual episodes
for key in keys:
    print(key)
    df = pd.read_sql_query(f"SELECT * from {key}", conn)
    for i, row in df.iterrows():
        poster_url = row['poster_url']
        if poster_url != "N/A":
            # insert poster
            img = requests.get(row['poster_url'], stream=True)
            cmd = f"UPDATE {key} SET poster = ? WHERE imdb_id = '{row['imdb_id']}'"
            curs.execute(cmd, [sqlite3.Binary(img.content)])
            conn.commit()

tos
tas
tng
ds9
voy
ent
dis
sho
pic
lds
pro
snw


## Movies Request

In [9]:
# all movies
st_movies = {'TMP':  'tt0079945',
             'TWOK': 'tt0084726',
             'TSFS': 'tt0088170',
             'TVH':  'tt0092007',
             'TFF':  'tt0098382',
             'TUC':  'tt0102975',
             'GEN':  'tt0111280',
             'FC':   'tt0117731',
             'INS':  'tt0120844',
             'NEM':  'tt0253754',
             'ST09': 'tt12126100',
             'STID': 'tt1408101',
             'STB':  'tt2660888',
             }

In [23]:
conn.execute(f"CREATE TABLE IF NOT EXISTS mov (abb TEXT, title TEXT, year TEXT, rated TEXT, released DATE, runtime TEXT, director TEXT, writer TEXT, actors TEXT, plot TEXT, poster_url TEXT, poster BLOB, metascore INTEGER, imdb_rating FLOAT, imdb_votes INTEGER, box_office TEXT, imdb_id TEXT)")

for key in st_movies.keys():
    res = omdb.imdbid(st_movies[key])

    # check if it is already in the database
    df_check = pd.read_sql_query(f"SELECT * FROM mov WHERE imdb_id = '{res['imdb_id']}'", conn)
    mov_exists = True if len(df_check) > 0 else False
    if not mov_exists:
        cmd = "INSERT INTO mov (abb, title, year, rated, released, runtime, director, writer, actors, plot, poster_url, metascore, imdb_rating, imdb_votes, imdb_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?) "
        curs.execute(cmd, [key, res['title'], res['year'], res['rated'], res['released'], res['runtime'], res['director'], res['writer'], res['actors'], res['plot'], res['poster'], res['metascore'], res['imdb_rating'], res['imdb_votes'], res['imdb_id']])
        conn.commit()
df = pd.read_sql_query("SELECT * FROM mov", conn)
df

,abb,title,year,rated,released,runtime,director,writer,actors,plot,poster_url,poster,metascore,imdb_rating,imdb_votes,box_office,imdb_id
0,TMP,Star Trek: The Motion Picture,1979,G,08 Dec 1979,132 min,Robert Wise,"Gene Roddenberry, Harold Livingston, Alan Dean...","William Shatner, Leonard Nimoy, DeForest Kelley",When an alien spacecraft of enormous power is ...,https://m.media-amazon.com/images/M/MV5BNjk1Zj...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,48,6.4,"88,419",None,tt0079945
1,TWOK,Star Trek II: The Wrath of Khan,1982,PG,04 Jun 1982,113 min,Nicholas Meyer,"Gene Roddenberry, Harve Bennett, Jack B. Sowards","William Shatner, Leonard Nimoy, DeForest Kelley","With the assistance of the Enterprise crew, Ad...",https://m.media-amazon.com/images/M/MV5BNmZiZm...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,67,7.7,"119,243",None,tt0084726
2,TSFS,Star Trek III: The Search for Spock,1984,PG,01 Jun 1984,105 min,Leonard Nimoy,"Gene Roddenberry, Harve Bennett","William Shatner, Leonard Nimoy, DeForest Kelley",Admiral Kirk and his bridge crew risk their ca...,https://m.media-amazon.com/images/M/MV5BMmFiMT...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,56,6.6,"79,449",None,tt0088170
3,TVH,Star Trek IV: The Voyage Home,1986,PG,26 Nov 1986,119 min,Leonard Nimoy,"Gene Roddenberry, Leonard Nimoy, Harve Bennett","William Shatner, Leonard Nimoy, DeForest Kelley","To save Earth from an alien probe, Admiral Jam...",https://m.media-amazon.com/images/M/MV5BZmU3ZG...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,71,7.3,"84,085",None,tt0092007
4,TFF,Star Trek V: The Final Frontier,1989,PG,09 Jun 1989,107 min,William Shatner,"Gene Roddenberry, William Shatner, Harve Bennett","William Shatner, Leonard Nimoy, DeForest Kelley",Captain Kirk and his crew must deal with Mr. S...,https://m.media-amazon.com/images/M/MV5BZmUzNj...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,43,5.5,"59,389",None,tt0098382
5,TUC,Star Trek VI: The Undiscovered Country,1991,PG,06 Dec 1991,110 min,Nicholas Meyer,"Gene Roddenberry, Leonard Nimoy, Lawrence Konner","William Shatner, Leonard Nimoy, DeForest Kelley","On the eve of retirement, Kirk and McCoy are c...",https://m.media-amazon.com/images/M/MV5BYzJiZD...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,65,7.2,"74,829",None,tt0102975
6,GEN,Star Trek: Generations,1994,PG,18 Nov 1994,118 min,David Carson,"Gene Roddenberry, Rick Berman, Ronald D. Moore","Patrick Stewart, William Shatner, Malcolm McDo...",With the help of long presumed dead Captain Ki...,https://m.media-amazon.com/images/M/MV5BOTk5Mj...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,55,6.6,"80,920",None,tt0111280
7,FC,Star Trek: First Contact,1996,PG-13,22 Nov 1996,111 min,Jonathan Frakes,"Gene Roddenberry, Rick Berman, Brannon Braga","Patrick Stewart, Jonathan Frakes, Brent Spiner",The Borg travel back in time intent on prevent...,https://m.media-amazon.com/images/M/MV5BYzMzZm...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,71,7.6,"123,981",None,tt0117731
8,INS,Star Trek: Insurrection,1998,PG,11 Dec 1998,103 min,Jonathan Frakes,"Gene Roddenberry, Rick Berman, Michael Piller","Patrick Stewart, Jonathan Frakes, Brent Spiner",When the crew of the Enterprise learn of a Fed...,https://m.media-amazon.com/images/M/MV5BNWEzZD...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,64,6.4,"74,493",None,tt0120844
9,NEM,Star Trek: Nemesis,2002,PG-13,13 Dec 2002,116 min,Stuart Baird,"Gene Roddenberry, John Logan, Rick Berman","Patrick Stewart, Jonathan Frakes, Brent Spiner",The Enterprise is diverted to the Romulan home...,https://m.media-amazon.com/images/M/MV5BMjAxNj...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,51,6.4,"78,345",None,tt0253754


## Download Movie Posters

In [11]:
# Download posters for series only
df = pd.read_sql_query(f"SELECT * from mov", conn)
for i, row in df.iterrows():
    poster_url = row['poster_url']
    if poster_url != "N/A":
        # insert poster
        img = requests.get(row['poster_url'], stream=True)
        cmd = f"UPDATE mov SET poster = ? WHERE imdb_id = '{row['imdb_id']}'"
        curs.execute(cmd, [sqlite3.Binary(img.content)])
        conn.commit()

## Build User Database

In [15]:
# delete current database if exists
db_filename = "user.db"
if os.path.exists(db_filename):
    os.remove(db_filename)

# make omdb requests for series
conn = sqlite3.connect(db_filename)
curs = conn.cursor()

conn.execute("DROP TABLE IF EXISTS log")
conn.execute("CREATE TABLE log (imdb_id TEXT, watched BOOLEAN, last_watched DATE, notes TEXT, favorite BOOLEAN, rating INTEGER, emoji TEXT)")
#conn.execute("INSERT INTO log (imdb_id, watched, last_watched) VALUES ('tt0708469', 1, '2022-02-04')")
conn.commit()

In [13]:
df = pd.read_sql_query("SELECT * FROM log", conn)
df

,imdb_id,watched,last_watched,notes,favorite,rating,emoji


In [14]:
bool(df[df["imdb_id"] == 'tt0708469']['watched'].values[0])

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
print(os.environ.keys())

In [ ]:
from appdirs import *

In [ ]:
user_data_dir('trekList')